In [12]:
# selecting aoi of interst from shape file:
# its all about understanding geopandas and its work
import geopandas as gpd
shp_path=r"C:\Users\Acer\Desktop\miniproject\data\data2\Districts.shp"
nepal=gpd.read_file(shp_path)
chitwan=nepal[nepal['DIST_NAME']=="Chitawan"]
chitwan=chitwan.to_crs(epsg=32645)
chitwan.crs




<Projected CRS: EPSG:32645>
Name: WGS 84 / UTM zone 45N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 84°E and 90°E, northern hemisphere between equator and 84°N, onshore and offshore. Bangladesh. Bhutan. China. India. Kazakhstan. Mongolia. Nepal. Russian Federation.
- bounds: (84.0, 0.0, 90.0, 84.0)
Coordinate Operation:
- name: UTM zone 45N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [13]:
import geemap

import ee
geemap.ee_initialize()

In [14]:
# it's all about changing gdf object to ee object becz we are  changing earthengine dataset sentinal 2 for ndbi
chitwan_only=geemap.gdf_to_ee(chitwan)
chitwan_area=chitwan_only.geometry()
Map=geemap.Map()


In [21]:
Map

Map(center=[27.58469920634319, 84.43650321937417], controls=(WidgetControl(options=['position', 'transparent_b…

In [15]:
# each image ko ndbi calculate garni function
def add_ndbi(image):
  nir=image.select('B8')
  swir=image.select('B11')
  # upscaling swir to 10m using billear resampling
  swir_resample=swir.resample('bilinear').reproject(
    crs=nir.projection(),
    scale=10
  )
  ndbi=swir_resample.subtract(nir).divide(swir_resample.add(nir)).rename('NDBI')
  return image.addBands(ndbi)


In [5]:
# this are the function which can be used for masking clouds 

# def mask_s2_clouds(image):
#     """
#     Correct cloud masking for Sentinel-2 using SCL band
#     SCL values:
#     0: No data
#     1: Saturated or defective
#     2: Dark area pixels
#     3: Cloud shadows
#     4: Vegetation
#     5: Bare soils
#     6: Water
#     7: Unclassified
#     8: Cloud medium probability
#     9: Cloud high probability
#     10: Thin cirrus
#     11: Snow or ice
#     """
#     scl = image.select("SCL")
    
#     # Mask out clouds, cloud shadows, and other bad pixels
#     # Keep only good quality pixels (vegetation, bare soil, water, unclassified)
#     good_pixels = scl.eq(4).Or(scl.eq(5)).Or(scl.eq(6)).Or(scl.eq(7))
    
    # return image.updateMask(good_pixels)
# def mask_s2_clouds(image):
#     """Masks clouds using the 'MSK_CLDPRB' band."""
#     clouds = image.select('MSK_CLDPRB').gt(65) # Greater than 65% cloud probability
#     return image.updateMask(clouds.Not())
def mask_s2_clouds(image):
    scl = image.select('SCL')
    cloud_mask = scl.eq(8).Or(scl.eq(9))  # Clouds medium/high probability
    prob_mask = image.select('MSK_CLDPRB').gt(50)
    combined_mask = cloud_mask.Or(prob_mask)
    return image.updateMask(combined_mask.Not())


In [ ]:
# def compute_ndbi_for_year(year,chitwan_area):
#   start_date=f'{year}-01-01'
#   end_date=f'{year}-12-31'
#   sentinel2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
#                   .filterDate(start_date, end_date)
#                   .filterBounds(chitwan_area)
#                   .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)))  
#   masked_image=sentinel2.map(mask_s2_clouds)
#   ndbi_collection=masked_image.map(add_ndbi)

#   # create a median composite from the ndvi collection
#   ndbi_composite=ndbi_collection.select('NDBI').median().clip(chitwan_area)
#   composite_band_names = ndbi_composite.bandNames()
#   print(f"Band names of NDVI composite for year {year}: {composite_band_names.getInfo()} ")
#   return ndbi_composite
# year=2020
# yearly_ndvi_composites_dict = {}
# print("proccesing year")
# ndbi_composite=compute_ndbi_for_year(year,chitwan_area)
# print(f"finish processing {year}")
# yearly_ndvi_composites_dict[year] = ndbi_composite




proccesing year
Band names of NDVI composite for year 2020: ['NDBI'] 
finish processing 2020


In [16]:
def compute_ndbi_for_year(year, chitwan_area):
    print(f"\n=== Processing year {year} ===")
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'
    
    # Get Sentinel-2 collection
    sentinel2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                  .filterDate(start_date, end_date)
                  .filterBounds(chitwan_area)
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)))
    
    print(f"Total images in collection: {sentinel2.size().getInfo()}")
    
    if sentinel2.size().getInfo() == 0:
        print("WARNING: No Sentinel-2 images found for this time period and area!")
        return None
    
    # Apply cloud masking
    masked_image = sentinel2.map(mask_s2_clouds)
    print(f"Images after cloud masking: {masked_image.size().getInfo()}")
   
    
    
    
    # Calculate NDBI
    ndbi_collection = masked_image.map(add_ndbi)
    print(f"NDBI collection size: {ndbi_collection.size().getInfo()}")

    
    # Create median composite
    ndbi_composite = ndbi_collection.select('NDBI').median().clip(chitwan_area)
    
    # Check if composite has data
    composite_stats = ndbi_composite.reduceRegion(
        reducer=ee.Reducer.minMax(),
        
        geometry=chitwan_area,
        scale=10,
        maxPixels=1e9
    )
    
    stats = composite_stats.getInfo()
    print(f"NDBI composite statistics: {stats}")
    
    composite_band_names = ndbi_composite.bandNames()
    print(f"Band names: {composite_band_names.getInfo()}")
    
    return ndbi_composite



In [17]:

# Test the function
years = range(2019,2024)
ndbi_dict={}
for year in years: 
    print("Starting NDBI computation...")
    ndbi_composite = compute_ndbi_for_year(year, chitwan_area)

    print(f"\nNDBI composite for {year} created successfully!")
    ndbi_dict[year]=ndbi_composite
    
    # Check if we have valid data
    # sample_value = ndbi_composite.sample(region=chitwan_area, scale=10, numPixels=1)
    # print(f"Sample NDBI value: {sample_value.getInfo()}")
    
    # Create map and add layer
   
   


Starting NDBI computation...

=== Processing year 2019 ===
Total images in collection: 54
Images after cloud masking: 54
NDBI collection size: 54
NDBI composite statistics: {'NDBI_max': 0.45957446098327637, 'NDBI_min': -0.5498444437980652}
Band names: ['NDBI']

NDBI composite for 2019 created successfully!
Starting NDBI computation...

=== Processing year 2020 ===
Total images in collection: 49
Images after cloud masking: 49
NDBI collection size: 49
NDBI composite statistics: {'NDBI_max': 0.4966363310813904, 'NDBI_min': -0.5595238208770752}
Band names: ['NDBI']

NDBI composite for 2020 created successfully!
Starting NDBI computation...

=== Processing year 2021 ===
Total images in collection: 54
Images after cloud masking: 54
NDBI collection size: 54
NDBI composite statistics: {'NDBI_max': 0.4527660310268402, 'NDBI_min': -0.6137953996658325}
Band names: ['NDBI']

NDBI composite for 2021 created successfully!
Starting NDBI computation...

=== Processing year 2022 ===
Total images in col

In [18]:
Map = geemap.Map()
ndbi_vis_params = {
    'min': -0.6,  # The actual minimum NDBI value
    'max': 0.4,   # The actual maximum NDBI value
    'palette': ['green', 'white', 'brown']  # Emphasize vegetation
    }
  


for year, ndbi_composite in ndbi_dict.items():   
    Map.addLayer(ndbi_composite, ndbi_vis_params, f'NDBI {year}')

    Map.centerObject(chitwan_area, 10)
    


In [19]:
Map.addLayer(chitwan_area, {}, 'Chitwan Area')

In [20]:
Map.add_legend(
    title="NDBI (Built-up Index)",
    legend_dict={
        "Vegetation (Low NDBI)": "green",
        "Mixed/Transitional": "white",
        "Built-up (High NDBI)": "brown"
    },
    position="topright" # You can change the position (e.g., "topright")
)
Map

Map(center=[27.58469920634309, 84.4365032193742], controls=(WidgetControl(options=['position', 'transparent_bg…